# Cost-Sensitive-Classification

Original notebook: https://github.com/richardangell/Python-Examples/blob/master/Cost-Sensitive-Classification.ipynb

In [ ]:
# Import local bentoml repository
import sys
import os
sys.path.insert(0, os.path.join(os.getcwd(), "..", ".."))
import bentoml

print(bentoml.__file__)

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn import metrics
import os
import zipfile
import warnings
import sklearn.exceptions


In [ ]:
!curl -i https://github.com/somedadaism/CreditCardDefault/raw/master/data/UCI_Credit_Card.csv -o UCI_Credit_Card.csv

In [ ]:
dataset = os.getcwd() + '/UCI_Credit_Card.csv'

In [ ]:
credit_card = pd.read_csv(dataset)
credit_card.head()

In [ ]:
fraud_cost_matrix = np.array([[0, 1], [10, 0]])
tn_cost, fp_cost, fn_cost, tp_cost = fraud_cost_matrix.ravel()
credit_card['weights'] = 0
credit_card.loc[credit_card['default.payment.next.month'] == 0, 'weights'] = fp_cost
credit_card.loc[credit_card['default.payment.next.month'] == 1, 'weights'] = fn_cost

#### Create sample column

In [ ]:
np.random.seed(1)
credit_card['samp'] = np.random.randint(low = 0, high = 6, size = credit_card.shape[0])
credit_card['samp'].value_counts().sort_index()


In [ ]:
weights_col = 'weights'
response_col = 'default.payment.next.month'
not_explanatory_cols = [response_col, weights_col] + \
                       ['ID', 'samp']

### Create Light GBM datasets

In [ ]:
explanatory_cols = list(set(credit_card.columns.values) - set(not_explanatory_cols))
explanatory_cols.sort()
len(explanatory_cols)

In [ ]:
train_expl = credit_card.loc[credit_card.samp < 4, explanatory_cols]
val_expl = credit_card.loc[credit_card.samp == 4, explanatory_cols]
test_expl = credit_card.loc[credit_card.samp == 5, explanatory_cols]

train_w = credit_card.loc[credit_card.samp < 4, weights_col]
val_w = credit_card.loc[credit_card.samp == 4, weights_col]
test_w = credit_card.loc[credit_card.samp == 5, weights_col]

train_resp = credit_card.loc[credit_card.samp < 4, response_col]
val_resp = credit_card.loc[credit_card.samp == 4, response_col]
test_resp = credit_card.loc[credit_card.samp == 5, response_col]

In [ ]:
lgb_train = lgb.Dataset(data = train_expl, label = train_resp)
lgb_val = lgb.Dataset(data = val_expl, label = val_resp)
lgb_test = lgb.Dataset(data = test_expl, label = test_resp)

lgb_train_w = lgb.Dataset(data = train_expl, label = train_resp, weight = train_w)
lgb_val_w = lgb.Dataset(data = val_expl, label = val_resp, weight = val_w)
lgb_test_w = lgb.Dataset(data = test_expl, label = test_resp, weight = test_w)

#### Build model

In [ ]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_leaves': 31,
    'learning_rate': 0.09,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5
}
evals_result_1 = {}
gbm_1 = lgb.train(params,
                  lgb_train,
                  num_boost_round = 100,
                  early_stopping_rounds = 5,
                  valid_sets = [lgb_train, lgb_val],
                  verbose_eval = False,
                  evals_result = evals_result_1)

In [ ]:
print('best iteration:', gbm_1.best_iteration)
lgb.plotting.plot_metric(evals_result_1)

#### Get prediction

In [ ]:
val_pred = gbm_1.predict(val_expl, num_iteration = gbm_1.best_iteration)
pd.Series(val_pred).plot('hist', bins = 50)


In [ ]:
type(val_expl)

# Export model using BentoML

In [ ]:
%%writefile lightgbm_classification_service.py

import pandas as pd
import bentoml
from bentoml.artifact import LightGBMArtifact
from bentoml.handlers import DataframeHandler

@bentoml.artifacts([LightGBMArtifact('gbm')])
@bentoml.env(conda_dependencies=["lightgbm", "pandas"])
class LightGBMClassificationModel(bentoml.BentoService):

    @bentoml.api(DataframeHandler, typ='series')
    def predict(self, df):
        """
        predict expects pandas.DataFrame as input
        """        
        return self.artifacts.gbm.predict(df)

In [ ]:
from lightgbm_classification_service import LightGBMClassificationModel

# Initialize bentoML model with artifacts

bento_model = LightGBMClassificationModel.pack(
    gbm=gbm_1
)

# Save bentoML model to directory
saved_path = bento_model.save("/tmp/bento")

# print the directory containing exported model archive (prefixed with model name and version)
print(saved_path)